In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

2024-10-15 06:48:38.247017: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 06:48:38.257747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 06:48:38.271098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 06:48:38.274634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 06:48:38.284410: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data = pd.read_csv('/home/hthnh/Desktop/project-khkt-thcs/model/processed_fit.csv')

In [3]:
X = data[['Co', 'VOC', 'Temp', 'Hum']]
y = data['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
base_model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification (normal vs smoke)
])

/home/hthnh/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1728949729.829881    9915 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-15 06:48:49.852899: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Ski

In [5]:
base_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])



In [6]:
base_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 2s 735us/step - accuracy: 0.9833 - loss: 0.0816 - val_accuracy: 0.9988 - val_loss: 0.0032
Epoch 2/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 674us/step - accuracy: 0.9989 - loss: 0.0031 - val_accuracy: 0.9992 - val_loss: 0.0021
Epoch 3/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 670us/step - accuracy: 0.9993 - loss: 0.0018 - val_accuracy: 0.9992 - val_loss: 0.0021
Epoch 4/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 628us/step - accuracy: 0.9993 - loss: 0.0017 - val_accuracy: 0.9993 - val_loss: 0.0022
Epoch 5/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step - accuracy: 0.9993 - loss: 0.0017 - val_accuracy: 0.9994 - val_loss: 0.0022
Epoch 6/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 594us/step - accuracy: 0.9993 - loss: 0.0016 - val_accuracy: 0.9994 - val_loss: 0.0015
Epoch 7/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 604us/step - accuracy: 0.9995 - loss: 0.0013 - val_accuracy: 0.9995 - val_loss: 0.0015
Epoch 8/20
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 1s 625us/step - accuracy: 0.9994 -

In [7]:
base_model.save('base_smoke_model.keras')

In [23]:
from tensorflow.keras.models import load_model
base_model = load_model('base_smoke_model.keras')

In [19]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
# Example new sensor readings (CO, VOC, Temp, Hum)
new_data = np.array([[40, 450, 30.2, 72.5]])  # Example values for testing
scaler = joblib.load('/home/hthnh/Desktop/project-khkt-thcs/model/scaler.pkl')
# Assuming you used a scaler for normalization during training
new_data_scaled = scaler.transform(new_data)  # Normalize the new data


/home/hthnh/.local/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
# Make predictions
prediction = base_model.predict(new_data_scaled)

# Interpret the prediction (assuming binary classification: 0 = normal, 1 = smoke)
if prediction[0][0] > 0.5:
    print("Smoke detected")
else:
    print("Normal environment")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Smoke detected
